# Regularität

In [1]:
from netgen.geom2d import Rectangle, CSG2d
from ngsolve import *
from ngsolve.webgui import Draw

In [2]:
import numpy as np

In [3]:
r1 = Rectangle((-1,-1),(1,1)).BC('bd')
r2 = Rectangle((-1.5,-1.5),(0,0)).BC('bd')
r3 = r1-r2

In [4]:
geo = CSG2d()
geo.Add(r3)

In [21]:
ngmesh = geo.GenerateMesh(maxh=0.5)
mesh = Mesh(ngmesh)

In [22]:
Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2104-121-gee6902d12', 'mesh_dim': 2, 'order2d': 1, 'order3d': 1, '…

BaseWebGuiScene

In [23]:
mesh.GetBoundaries()

('bd', 'bd', 'bd', 'bd', 'bd', 'bd')

In [108]:
V.ndof

1361

In [109]:
V = H1(mesh,order=4,dirichlet='bd')
print('free dofs = ', V.ndof)
u,v = V.TnT()
gfu = GridFunction(V)

free dofs =  1361


In [110]:
a = BilinearForm(V)
a += grad(u)*grad(v)*dx
a.Assemble()

f = LinearForm(V)
f += CoefficientFunction(1)*v*dx
f.Assemble()

In [111]:
r = sqrt(x**2+y**2)
theta = atan2(y,x)
ud = r**(2/3)*sin((2*theta+np.pi)/3)

In [112]:
gfu.Set(ud,definedon=mesh.Boundaries('bd'))

In [113]:
b = f.vec.CreateVector()
b.data = f.vec -a.mat*gfu.vec

In [114]:
gfu.vec.data += a.mat.Inverse(freedofs=V.FreeDofs())*b

In [115]:
Draw(gfu)

WebGuiWidget(value={'ngsolve_version': '6.2.2104-121-gee6902d12', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

BaseWebGuiScene

In [116]:
res = gfu-ud

In [117]:
Draw(res,mesh,'res')

WebGuiWidget(value={'ngsolve_version': '6.2.2104-121-gee6902d12', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

BaseWebGuiScene

In [118]:
space_flux = HDiv(mesh, order=2)
gf_flux = GridFunction(space_flux, "flux")

flux = grad(gfu)
gf_flux.Set(flux)

In [94]:
err = (flux-gf_flux)*(flux-gf_flux)
Draw(err, mesh, 'error_representation')

WebGuiWidget(value={'ngsolve_version': '6.2.2104-121-gee6902d12', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

BaseWebGuiScene

In [95]:
eta2 = Integrate(err, mesh, VOL, element_wise=True)
print(eta2)

 9.4778e-06
 1.17756e-05
 1.42816e-05
 4.30034e-06
 1.61514e-07
 0.000115226
 7.86774e-06
 1.01349e-07
 1.61514e-07
 2.59774e-08
 1.17756e-05
 1.42816e-05
 2.59774e-08
 9.4778e-06
 0.000115226
 2.71155e-07
 1.01349e-07
 0.000619801
 4.30034e-06
 0.000765462
 0.000765462
 0.000619801
 1.55524e-08
 9.44418e-09
 4.77892e-07
 4.5826e-08
 2.70746e-08
 5.96086e-09
 4.17535e-06
 1.09328e-08
 4.17535e-06
 1.09328e-08
 9.44418e-09
 1.55524e-08
 8.80199e-09
 4.671e-09
 2.35581e-08
 1.3032e-08
 2.20394e-07
 1.64445e-08
 1.06945e-08
 2.42478e-09
 1.3032e-08
 2.35581e-08
 4.671e-09
 8.80198e-09
 3.77905e-06
 3.77905e-06
 5.96085e-09
 2.70746e-08
 4.5826e-08
 4.77892e-07
 2.68737e-06
 2.68737e-06
 1.00645e-08
 3.53452e-09
 4.10687e-09
 4.04534e-09
 4.53352e-09
 2.04822e-09
 4.04534e-09
 4.10687e-09
 1.44537e-08
 6.90264e-09
 3.50488e-09
 3.50488e-09
 2.04823e-09
 4.53352e-09
 3.53452e-09
 1.00645e-08
 4.4575e-07
 4.4575e-07
 2.71155e-07
 6.90265e-09
 1.44537e-08
 3.79042e-09
 3.79042e-09
 7.86774e-0

In [96]:
maxerr = max(eta2)
print ("maxerr = ", maxerr)

for el in mesh.Elements():
    mesh.SetRefinementFlag(el, eta2[el.nr] > 0.25*maxerr)

maxerr =  0.0007654622249502114


In [97]:
mesh.Refine()

In [98]:
Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2104-121-gee6902d12', 'mesh_dim': 2, 'order2d': 1, 'order3d': 1, '…

BaseWebGuiScene